In [114]:
# --{INSTALAR E IMPORTAR LIBRERIAS PARA CONECTARSE AL SERVIDOR mySQL}--
!pip install mysql-connector-python

import mysql.connector # !pip install mysql-connector-python 
from mysql.connector import Error
import warnings 
warnings.filterwarnings('ignore')

You should consider upgrading via the 'c:\users\eljl2\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [115]:
# --{COMPROBAR CONEXION CON EL SERVIDOR mySQL}--
try: 
    connection = mysql.connector.connect(host='127.0.0.1', 
                                         database='pedidos',
                                         user='jlVV',
                                         password='16042510', 
                                         connection_timeout=180)

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL database... MySQL Server version on ", db_Info)

        cursor = connection.cursor()
        connection.commit()

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    # closing database connection.
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL database... MySQL Server version on  8.0.40
MySQL connection is closed


### EMPEZAMOS A CONECTARNOS Y EXTRAER INFO : 

In [116]:
# --{ESTABLECER CONEXIÓN Y CONECTAR EL CURSOR}--
connection = mysql.connector.connect(host='127.0.0.1', 
                                     database='pedidos',
                                     user='jlVV',
                                     password='16042510', 
                                     connection_timeout=180000)

# Conectar el cursor, objeto importante para facilitar la ejecución de los comandos necesarios
cursor = connection.cursor()

**1. Obtener el listado de todos los productos**

In [117]:
# --{OBTENER TODOS LOS PRODUCTOS}--
query1 = """ SELECT * FROM PRODUCTO; """  
cursor.execute(query1)
cursor.fetchall()

[('01', 'Hamburguesa', 2.6),
 ('02', 'Patatas', 2.0),
 ('03', 'tomate', 0.5),
 ('04', 'Queso', 1.0),
 ('05', 'Lechuga', 0.5),
 ('06', 'Pollo', 3.6),
 ('08', 'Bacon', 1.5),
 ('09', 'Coca cola', 3.0),
 ('10', 'Nestea', 3.0),
 ('11', 'Menú de Pollo', 5.0),
 ('12', 'Menú de Hamburguesa con queso', 6.0),
 ('13', 'Menú de Pollo con queso', 6.0),
 ('14', 'Menú de Hamburguesa', 6.0),
 ('15', 'Helado', 1.0),
 ('16', 'Tarta', 2.0),
 ('17', 'Fruta', 1.0),
 ('18', 'Agua', 2.0),
 ('19', 'Fanta', 3.0)]

**2. Obtener el número de pedido, dni del repartidor y hora de reparto de aquellos pedidos que se toman nota después de las siete de la tarde**

In [118]:
query2 = """ SELECT Numero, DNI_R, Hora_rep FROM PEDIDO WHERE Hora_tm >'19:00' and Hora_rep is not null; """
cursor.execute(query2)
cursor.fetchall()

[('0005', '14188151T', datetime.timedelta(seconds=71100)),
 ('0006', '04477744T', datetime.timedelta(seconds=78300)),
 ('0007', '14188151T', datetime.timedelta(seconds=84900))]

**3. Obtener todos los campos de empleados que cobran entre 900 y 1000 euros.**


In [119]:
query3 = """ SELECT * FROM EMPLEADO WHERE Salario BETWEEN 900 AND 1000;  """
cursor.execute(query3)
cursor.fetchall()

[('04444444T', 'Úrsula Delta Camacho', '11177788899', 'mañana', 900.0),
 ('11111111Q', 'Luis Ramírez Pardo', '23445556666', 'mañana', 900.0),
 ('14111155T', 'Sergio Lérida Campos', '55577700089', 'tarde', 1000.0),
 ('22222222S', 'María Sánchez Cid', '11112223334', 'tarde', 1000.0),
 ('33333333M', 'Martín Guerrero López', '33344455566', 'tarde', 1000.0),
 ('55555555J', 'Carmen Hernández Pío', '99966633311', 'mañana', 900.0),
 ('77777777M', 'Pedro Jiménez Ruiz', '23456785432', 'tarde', 1000.0),
 ('99999999X', 'Raúl Rodrigo Roca', '55566677789', 'tarde', 1000.0)]

**4. Obtener el número de pedido e importe de aquellos que han sido registrados en el mes de noviembre de 2020 y su importe es mayor a 15 Euros.**

In [120]:
query4 = """ SELECT Numero, importe FROM PEDIDO WHERE fecha BETWEEN '2020-11-01' AND '2020-11-30' AND importe >'15'"""
cursor.execute(query4)
cursor.fetchall()

[('0006', 23.0), ('0009', 25.0), ('0010', 45.0), ('0011', 45.0)]

**5. Obtener por cada uno de los repartidores su DNI junto a la cantidad de pedidos entregados.**

In [121]:
query5 = """ SELECT DNI_R, COUNT(*) AS TOTAL_PEDIDOS_REPARTIDOS FROM PEDIDO where dni_R is not null GROUP BY DNI_R; """
cursor.execute(query5)
cursor.fetchall()

[('04477744T', 2),
 ('11245621Q', 1),
 ('14188151T', 2),
 ('55675675J', 1),
 ('99009900J', 3)]

**6. Obtener por cada mes (con el formato de nombre y no de número ej. Noviembre),la cantidad de pedidos realizados**

In [122]:
query6 = """ SELECT DNI_R, COUNT(*) AS TOTAL_PEDIDOS_REPARTIDOS FROM PEDIDO where dni_R is not null GROUP BY DNI_R; """ 
cursor.execute(query6)
cursor.fetchall()

[('04477744T', 2),
 ('11245621Q', 1),
 ('14188151T', 2),
 ('55675675J', 1),
 ('99009900J', 3)]

**7. Obtener por cada mes (con el formato de nombre y no de número ej. Noviembre),la cantidad de pedidos realizados.**

In [123]:
query7 = """ SELECT 
    MONTHNAME(fecha) as Mes, 
    COUNT(Numero) as Cant_pedidos 
FROM 
    PEDIDO P 
WHERE 
    P.Numero IS NOT NULL 
    AND fecha IS NOT NULL 
GROUP BY 
    Mes;
 """
cursor.execute(query7)
cursor.fetchall()

[('October', 2), ('November', 6), ('September', 3)]

**8. Obtener un listado con el nombre y dni de los empleados en el mismo campo  </br> y
anteponiendo el dni al nombre de la siguiente forma </br>(ej; 45776633P, Juan Rodríguez López)
que tenga turno de “tarde” o "noche". Ordenar el listado por dni.**

In [124]:
query8 = """ SELECT CONCAT(DNI,', ', Nombre) as Identificación_completa FROM EMPLEADO E WHERE Turno IN ('tarde','noche') ORDER BY dni; """
cursor.execute(query8)
cursor.fetchall()

[('03232323P, María Luisa Galdón Ter',),
 ('14111155T, Sergio Lérida Campos',),
 ('14567555L, Piedad Colmenero Zapillo',),
 ('22222222S, María Sánchez Cid',),
 ('33333333M, Martín Guerrero López',),
 ('77777777M, Pedro Jiménez Ruiz',),
 ('88888888O, Soledad Campillo Molina',),
 ('99999999X, Raúl Rodrigo Roca',)]

**9. Obtener el nombre, código y precio de aquellos productos que superen o igualen la media de todos los precios. Ordenar de mayor a menor precio.**

In [125]:
query9 = """ SELECT nombre, codigo, precio FROM PRODUCTO WHERE precio >= (SELECT AVG(precio) FROM PRODUCTO) ORDER BY precio DESC  """
cursor.execute(query9)
cursor.fetchall()

[('Menú de Hamburguesa con queso', '12', 6.0),
 ('Menú de Pollo con queso', '13', 6.0),
 ('Menú de Hamburguesa', '14', 6.0),
 ('Menú de Pollo', '11', 5.0),
 ('Pollo', '06', 3.6),
 ('Coca cola', '09', 3.0),
 ('Nestea', '10', 3.0),
 ('Fanta', '19', 3.0)]

**10. Obtener un listado con el nombre y DNI de los empleados que no han preparado nunca ningún pedido.**

In [126]:
query10 = """ SELECT nombre, dni FROM EMPLEADO WHERE dni NOT IN (SELECT DISTINCT dni_EP FROM PEDIDO); """
cursor.execute(query10)
cursor.fetchall()

[('Luis Ramírez Pardo', '11111111Q'),
 ('Sergio Lérida Campos', '14111155T'),
 ('Raúl Rodrigo Roca', '99999999X')]

**11. Obtener el código, nombre y precio de los productos(estos dos últimos en el mismo </br>
campo) que están contenidos en los pedidos que ha tomado nota "Luis" o "María</br>
Luisa". Ordena el listado de mayor a menor valor por fecha del pedido**

In [127]:
query11 = """ SELECT PR.Codigo, CONCAT(PR.Nombre, ' ' , PR.Precio) AS NOMBRE_PRECIO
                FROM PRODUCTO PR, PEDIDO P, consta C, EMPLEADO E
                WHERE PR.Codigo = C.Codigo_Pr
                AND P.Numero= C.Numero_P
                AND P.DNI_ETM= E.DNI
                AND E.nombre LIKE '%Luis%'
                ORDER BY P.Fecha DESC """
                
cursor.execute(query11)
cursor.fetchall()

[('11', 'Menú de Pollo 5.00'),
 ('12', 'Menú de Hamburguesa con queso 6.00'),
 ('02', 'Patatas 2.00'),
 ('09', 'Coca cola 3.00'),
 ('12', 'Menú de Hamburguesa con queso 6.00')]

**12. Obtener por cada repartidor, su nombre, cantidad de pedidos y el tiempo medio </br> que tardan en entregar los pedidos una vez preparados ordenar el listado el </br> tiempo medio que tardan en entregarlos: </br>**


In [128]:
query12 = """ SELECT R.nombre, COUNT(*) AS cantidad_repartos,
TIME_FORMAT(AVG(TIMEDIFF(Hora_rep,Hora_pre)),'%T')AS tiempo_entrega
FROM REPARTIDOR R, PEDIDO P
WHERE R.DNI = P.DNI_R
GROUP BY R.DNI
ORDER BY 3; """
cursor.execute(query12)
cursor.fetchall()

[('Laura Jiménez Jiménez', 2, '00:12:50'),
 ('Juan Pardo Rubio', 1, '00:15:00'),
 ('Carlos Sánchez Ruíz', 2, '00:19:00'),
 ('Alejandro Pardo López', 3, '00:25:00'),
 ('Juan Sánchez López', 1, '00:30:00')]

**13. Obtener un listado obteniendo el código, nombre y el precio de los productos cuyo precios sea el más barato o el más caro de todos**

In [129]:
query13 = """ SELECT Codigo, Nombre, precio
FROM PRODUCTO
WHERE precio = (SELECT MAX(precio)
FROM PRODUCTO)
OR precio = (SELECT MIN(precio)
FROM PRODUCTO);
    """
cursor.execute(query13)
cursor.fetchall()

[('03', 'tomate', 0.5),
 ('05', 'Lechuga', 0.5),
 ('12', 'Menú de Hamburguesa con queso', 6.0),
 ('13', 'Menú de Pollo con queso', 6.0),
 ('14', 'Menú de Hamburguesa', 6.0)]

**14. Obtener por cada producto , el nombre y código el número total de pedidos en los que se encuentra teniendo en cuenta que el total de pedidos en los cuales se  </br> encuentre sea superior o igual a dos. Ordena el listado de mayor a menor número de productos.**

In [130]:
query14 = """ SELECT PR.nombre, PR.codigo, count(P.Numero) as cantidad_pedidos
FROM PEDIDO P, PRODUCTO PR, consta C
WHERE P.numero = C.numero_P
AND PR.codigo = C.codigo_Pr
GROUP BY PR.codigo
HAVING cantidad_pedidos>=2
ORDER BY cantidad_pedidos DESC; """
cursor.execute(query14)
cursor.fetchall()

[('Menú de Hamburguesa con queso', '12', 6),
 ('Menú de Pollo con queso', '13', 6),
 ('Menú de Pollo', '11', 3),
 ('Queso', '04', 3),
 ('Patatas', '02', 2),
 ('Helado', '15', 2)]

**15. Mostrar listado de los empleados (código y NSS en la misma columna) que han tomado nota de algún pedido y contienen el producto de código 13 y además el repartidor sea 'Laura'.**

In [131]:
query15 = """ SELECT E.nombre, CONCAT(E.DNI, ' ' , E.NSS) as Identificación
FROM EMPLEADO E, PEDIDO P, REPARTIDOR R, consta C
WHERE E.DNI = P.DNI_ETM AND R.DNI= P.DNI_R AND P.numero =
C.numero_P
AND R.nombre LIKE '%Laura%'
AND C.codigo_Pr ='13'; """
cursor.execute(query15)
cursor.fetchall()

[('Raúl Rodrigo Roca', '99999999X 55566677789'),
 ('Piedad Colmenero Zapillo', '14567555L 23456734534')]

**16. Obtener el nombre del producto que es menú junto con el código de los productos que lo componen en aquellos pedidos del mes de septiembre de 2020.**

In [132]:
query16 = """ SELECT P.numero AS numero_pedido,PR.nombre , PR.codigo,
EC.codigo_P_compuesto as
producto_que_lo_compone, PRODUCTO.nombre as descripción_p_que_lo_compone
FROM PRODUCTO PR, PEDIDO P, esta_compuesto EC, consta C, PRODUCTO PRODUCTO
WHERE PR.codigo=EC.codigo_p AND PR.codigo=C.codigo_PR
AND P.numero=C.numero_P AND PRODUCTO.codigo= EC.codigo_p_compuesto
AND P.fecha BETWEEN '2020-09-01' AND '2020-09-30'; """
cursor.execute(query16)
cursor.fetchall()

[('0005', 'Menú de Hamburguesa con queso', '12', '01', 'Hamburguesa'),
 ('0005', 'Menú de Hamburguesa con queso', '12', '02', 'Patatas'),
 ('0005', 'Menú de Hamburguesa con queso', '12', '04', 'Queso'),
 ('0005', 'Menú de Hamburguesa con queso', '12', '10', 'Nestea'),
 ('0005', 'Menú de Hamburguesa con queso', '12', '16', 'Tarta'),
 ('0005', 'Menú de Pollo con queso', '13', '02', 'Patatas'),
 ('0005', 'Menú de Pollo con queso', '13', '04', 'Queso'),
 ('0005', 'Menú de Pollo con queso', '13', '06', 'Pollo'),
 ('0005', 'Menú de Pollo con queso', '13', '08', 'Bacon'),
 ('0005', 'Menú de Pollo con queso', '13', '09', 'Coca cola'),
 ('0005', 'Menú de Pollo con queso', '13', '15', 'Helado'),
 ('0007', 'Menú de Hamburguesa con queso', '12', '01', 'Hamburguesa'),
 ('0007', 'Menú de Hamburguesa con queso', '12', '02', 'Patatas'),
 ('0007', 'Menú de Hamburguesa con queso', '12', '04', 'Queso'),
 ('0007', 'Menú de Hamburguesa con queso', '12', '10', 'Nestea'),
 ('0007', 'Menú de Hamburguesa con q

## Obtener tablas como df 

Los resultados de las consultas que hemos hechoa antes, las vamos a trasnformar en dataframes

In [133]:
import pandas as pd


**1. Obtener el listado de todos los productos**

In [134]:
df1 = pd.read_sql(query1, con=connection)
df1

,Codigo,Nombre,Precio
0,01,Hamburguesa,2.6
1,02,Patatas,2.0
2,03,tomate,0.5
3,04,Queso,1.0
4,05,Lechuga,0.5
5,06,Pollo,3.6
6,08,Bacon,1.5
7,09,Coca cola,3.0
8,10,Nestea,3.0
9,11,Menú de Pollo,5.0


**2. Obtener el número de pedido, dni del repartidor y hora de reparto de aquellos pedidos que se toman nota después de las siete de la tarde**

In [135]:
df2 = pd.read_sql(query2, con=connection)
df2

,Numero,DNI_R,Hora_rep
0,0005,14188151T,0 days 19:45:00
1,0006,04477744T,0 days 21:45:00
2,0007,14188151T,0 days 23:35:00


**3. Obtener todos los campos de empleados que cobran entre 900 y 1000 euros.**

In [136]:
df3 = pd.read_sql(query3, con=connection)
df3

,DNI,Nombre,Nss,Turno,Salario
0,04444444T,Úrsula Delta Camacho,11177788899,mañana,900.0
1,11111111Q,Luis Ramírez Pardo,23445556666,mañana,900.0
2,14111155T,Sergio Lérida Campos,55577700089,tarde,1000.0
3,22222222S,María Sánchez Cid,11112223334,tarde,1000.0
4,33333333M,Martín Guerrero López,33344455566,tarde,1000.0
5,55555555J,Carmen Hernández Pío,99966633311,mañana,900.0
6,77777777M,Pedro Jiménez Ruiz,23456785432,tarde,1000.0
7,99999999X,Raúl Rodrigo Roca,55566677789,tarde,1000.0


**4. Obtener el número de pedido e importe de aquellos que han sido registrados en el mes de noviembre de 2020 y su importe es mayor a 15 Euros.**

In [137]:
df4 = pd.read_sql(query4, con=connection)
df4

,Numero,importe
0,0006,23.0
1,0009,25.0
2,0010,45.0
3,0011,45.0


**5. Obtener por cada uno de los repartidores su DNI junto a la cantidad de pedidos entregados.**

In [138]:
df5 = pd.read_sql(query5, con=connection)
df5

,DNI_R,TOTAL_PEDIDOS_REPARTIDOS
0,04477744T,2
1,11245621Q,1
2,14188151T,2
3,55675675J,1
4,99009900J,3


**6. Obtener por cada mes (con el formato de nombre y no de número ej. Noviembre),la cantidad de pedidos realizados**

In [139]:
df6 = pd.read_sql(query6, con=connection)
df6

,DNI_R,TOTAL_PEDIDOS_REPARTIDOS
0,04477744T,2
1,11245621Q,1
2,14188151T,2
3,55675675J,1
4,99009900J,3


**7. Obtener por cada mes (con el formato de nombre y no de número ej. Noviembre),la cantidad de pedidos realizados.**

In [140]:
df7 = pd.read_sql(query7, con=connection)
df7

,Mes,Cant_pedidos
0,October,2
1,November,6
2,September,3


**8. Obtener un listado con el nombre y dni de los empleados en el mismo campo  </br> y
anteponiendo el dni al nombre de la siguiente forma </br>(ej; 45776633P, Juan Rodríguez López)
que tenga turno de “tarde” o "noche". Ordenar el listado por dni.**

In [141]:
df8 = pd.read_sql(query8, con=connection)
df8

,Identificación_completa
0,"03232323P, María Luisa Galdón Ter"
1,"14111155T, Sergio Lérida Campos"
2,"14567555L, Piedad Colmenero Zapillo"
3,"22222222S, María Sánchez Cid"
4,"33333333M, Martín Guerrero López"
5,"77777777M, Pedro Jiménez Ruiz"
6,"88888888O, Soledad Campillo Molina"
7,"99999999X, Raúl Rodrigo Roca"


**9. Obtener el nombre, código y precio de aquellos productos que superen o igualen la media de todos los precios. Ordenar de mayor a menor precio.**

In [142]:
df9 = pd.read_sql(query9, con=connection)
df9

,nombre,codigo,precio
0,Menú de Hamburguesa con queso,12,6.0
1,Menú de Pollo con queso,13,6.0
2,Menú de Hamburguesa,14,6.0
3,Menú de Pollo,11,5.0
4,Pollo,06,3.6
5,Coca cola,09,3.0
6,Nestea,10,3.0
7,Fanta,19,3.0


**10. Obtener un listado con el nombre y DNI de los empleados que no han preparado nunca ningún pedido.**

In [143]:
df10 = pd.read_sql(query10, con=connection)
df10

,nombre,dni
0,Luis Ramírez Pardo,11111111Q
1,Sergio Lérida Campos,14111155T
2,Raúl Rodrigo Roca,99999999X


**11. Obtener el código, nombre y precio de los productos(estos dos últimos en el mismo </br>
campo) que están contenidos en los pedidos que ha tomado nota "Luis" o "María</br>
Luisa". Ordena el listado de mayor a menor valor por fecha del pedido**

In [144]:
df11 = pd.read_sql(query11, con=connection)
df11

,Codigo,NOMBRE_PRECIO
0,11,Menú de Pollo 5.00
1,12,Menú de Hamburguesa con queso 6.00
2,02,Patatas 2.00
3,09,Coca cola 3.00
4,12,Menú de Hamburguesa con queso 6.00


**12. Obtener por cada repartidor, su nombre, cantidad de pedidos y el tiempo medio </br>
que tardan en entregar los pedidos una vez preparados. Ordenar el listado el </br>
tiempo medio que tardan en entregarlos: </br>**

In [145]:
df12 = pd.read_sql(query12, con=connection)
df12

,nombre,cantidad_repartos,tiempo_entrega
0,Laura Jiménez Jiménez,2,00:12:50
1,Juan Pardo Rubio,1,00:15:00
2,Carlos Sánchez Ruíz,2,00:19:00
3,Alejandro Pardo López,3,00:25:00
4,Juan Sánchez López,1,00:30:00


**13. Obtener un listado obteniendo el código, nombre y el precio de los productos cuyo precios sea el más barato o el más caro de todos**

In [146]:
df13 = pd.read_sql(query13, con=connection)
df13

,Codigo,Nombre,precio
0,03,tomate,0.5
1,05,Lechuga,0.5
2,12,Menú de Hamburguesa con queso,6.0
3,13,Menú de Pollo con queso,6.0
4,14,Menú de Hamburguesa,6.0


**14. Obtener por cada producto , el nombre y código el número total de pedidos en los que se encuentra teniendo en cuenta que el total de pedidos en los cuales se  </br> encuentre sea superior o igual a dos. Ordena el listado de mayor a menor número de productos.**

In [147]:
df14 = pd.read_sql(query14, con=connection)
df14

,nombre,codigo,cantidad_pedidos
0,Menú de Hamburguesa con queso,12,6
1,Menú de Pollo con queso,13,6
2,Menú de Pollo,11,3
3,Queso,04,3
4,Patatas,02,2
5,Helado,15,2


**15. Mostrar listado de los empleados (código y NSS en la misma columna) que han tomado nota de algún pedido y contienen el producto de código 13 y además el repartidor sea 'Laura'.**

In [148]:
df15 = pd.read_sql(query15, con=connection)
df15

,nombre,Identificación
0,Raúl Rodrigo Roca,99999999X 55566677789
1,Piedad Colmenero Zapillo,14567555L 23456734534


**16. Obtener por cada producto , el nombre y código el número total de pedidos en los que se encuentra teniendo en cuenta que el total de pedidos en los cuales se  </br>
encuentre sea superior o igual a dos. Ordena el listado de mayor a menor número de productos.**

In [149]:
df16 = pd.read_sql(query16, con=connection)
df16

,numero_pedido,nombre,codigo,producto_que_lo_compone,descripción_p_que_lo_compone
0,0005,Menú de Hamburguesa con queso,12,01,Hamburguesa
1,0005,Menú de Hamburguesa con queso,12,02,Patatas
2,0005,Menú de Hamburguesa con queso,12,04,Queso
3,0005,Menú de Hamburguesa con queso,12,10,Nestea
4,0005,Menú de Hamburguesa con queso,12,16,Tarta
5,0005,Menú de Pollo con queso,13,02,Patatas
6,0005,Menú de Pollo con queso,13,04,Queso
7,0005,Menú de Pollo con queso,13,06,Pollo
8,0005,Menú de Pollo con queso,13,08,Bacon
9,0005,Menú de Pollo con queso,13,09,Coca cola


**RESULTADOS AVANZADOS:**

In [150]:
# mostrando todas las columnas sin restricción
pd.set_option("display.max_columns", None, "display.max_rows", None, "display.max_colwidth", None)

**17. Muestra todas las tablas de la base de datos**

In [151]:
esquema = pd.read_sql("SHOW TABLES;", con=connection)
esquema

,Tables_in_pedidos
0,consta
1,empleado
2,esta_compuesto
3,pedido
4,pedidos_finalizados
5,producto
6,ranking_productos
7,repartidor


**18. Muestra toda la informacion de las tablas en el esquema de la base de datos**

In [152]:
esquema = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.Tables", con=connection)
esquema

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE,ENGINE,VERSION,ROW_FORMAT,TABLE_ROWS,AVG_ROW_LENGTH,DATA_LENGTH,MAX_DATA_LENGTH,INDEX_LENGTH,DATA_FREE,AUTO_INCREMENT,CREATE_TIME,UPDATE_TIME,CHECK_TIME,TABLE_COLLATION,CHECKSUM,CREATE_OPTIONS,TABLE_COMMENT
0,def,mysql,innodb_table_stats,BASE TABLE,InnoDB,10.0,Dynamic,33.0,496.0,16384.0,0.0,0.0,4194304.0,NaN,2025-01-29 12:08:12,2025-01-29 16:29:04,None,utf8mb3_bin,None,row_format=DYNAMIC stats_persistent=0,
1,def,mysql,innodb_index_stats,BASE TABLE,InnoDB,10.0,Dynamic,263.0,249.0,65536.0,0.0,0.0,4194304.0,NaN,2025-01-29 12:08:12,2025-01-29 16:29:04,None,utf8mb3_bin,None,row_format=DYNAMIC stats_persistent=0,
2,def,information_schema,CHARACTER_SETS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
3,def,information_schema,CHECK_CONSTRAINTS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
4,def,information_schema,COLLATIONS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
5,def,information_schema,COLLATION_CHARACTER_SET_APPLICABILITY,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
6,def,information_schema,COLUMNS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
7,def,information_schema,COLUMNS_EXTENSIONS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
8,def,information_schema,COLUMN_STATISTICS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,
9,def,information_schema,EVENTS,SYSTEM VIEW,None,10.0,None,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2025-01-29 13:08:15,NaT,None,None,None,,


**19. Muestra el nombre de todas las tablas**

In [153]:
tablas = pd.read_sql("SHOW TABLES", con=connection) 
tablas

,Tables_in_pedidos
0,consta
1,empleado
2,esta_compuesto
3,pedido
4,pedidos_finalizados
5,producto
6,ranking_productos
7,repartidor


**20. Muestra en una tabla todos registros de las tablas empleado, pedido y producto**

In [154]:
def consultasql(consulta): 
        connection = mysql.connector.connect(host='127.0.0.1', 
                                         database='pedidos',
                                         user='jlVV',
                                         password='16042510', 
                                         connection_timeout=180000)       
        SQL_Query = pd.read_sql_query(consulta,connection) 
        return SQL_Query
    
consultasql("select * from EMPLEADO")

,DNI,Nombre,Nss,Turno,Salario
0,03232323P,María Luisa Galdón Ter,43534534567,noche,1200.0
1,04444444T,Úrsula Delta Camacho,11177788899,mañana,900.0
2,11111111Q,Luis Ramírez Pardo,23445556666,mañana,900.0
3,14111155T,Sergio Lérida Campos,55577700089,tarde,1000.0
4,14567555L,Piedad Colmenero Zapillo,23456734534,noche,1200.0
5,22222222S,María Sánchez Cid,11112223334,tarde,1000.0
6,33333333M,Martín Guerrero López,33344455566,tarde,1000.0
7,55555555J,Carmen Hernández Pío,99966633311,mañana,900.0
8,77777777M,Pedro Jiménez Ruiz,23456785432,tarde,1000.0
9,88888888O,Soledad Campillo Molina,00088877754,noche,1200.0


In [155]:
consultasql("select * from PEDIDO")

,Numero,Fecha,Importe,DNI_ETM,DNI_EP,DNI_R,Hora_tm,Hora_pre,Hora_rep
0,0001,2020-10-15,10.0,11111111Q,04444444T,55675675J,0 days 12:00:00,0 days 12:15:00,0 days 12:45:00
1,0002,2020-11-11,15.0,22222222S,77777777M,99009900J,0 days 13:30:00,0 days 13:45:00,0 days 14:05:00
2,0003,2020-10-15,13.0,77777777M,22222222S,99009900J,0 days 15:00:00,0 days 15:15:00,0 days 15:45:00
3,0004,2020-11-10,13.0,99999999X,77777777M,04477744T,0 days 14:02:00,0 days 14:30:00,0 days 14:45:00
4,0005,2020-09-05,14.0,14567555L,03232323P,14188151T,0 days 19:02:00,0 days 19:30:00,0 days 19:45:00
5,0006,2020-11-15,23.0,14567555L,88888888O,04477744T,0 days 21:02:00,0 days 21:35:00,0 days 21:45:00
6,0007,2020-09-25,17.0,03232323P,88888888O,14188151T,0 days 23:05:00,0 days 23:12:00,0 days 23:35:00
7,0008,2020-09-15,14.0,99999999X,33333333M,11245621Q,0 days 18:02:00,0 days 18:30:00,0 days 18:45:00
8,0009,2020-11-23,25.0,04444444T,55555555J,99009900J,0 days 11:02:00,0 days 11:30:00,0 days 11:55:00
9,0010,2020-11-05,45.0,88888888O,14567555L,None,0 days 22:05:00,0 days 22:12:00,NaT


In [156]:
consultasql("select * from PRODUCTO")

,Codigo,Nombre,Precio
0,01,Hamburguesa,2.6
1,02,Patatas,2.0
2,03,tomate,0.5
3,04,Queso,1.0
4,05,Lechuga,0.5
5,06,Pollo,3.6
6,08,Bacon,1.5
7,09,Coca cola,3.0
8,10,Nestea,3.0
9,11,Menú de Pollo,5.0


**21. Crea un dataframe que contenga el dni, nombre, nss, truno y salario de todos los empleados y exportalo en un archivo .csv**

In [157]:
empleado = pd.DataFrame(empleado) 

In [158]:
empleado.head()

,DNI,Nombre,Nss,Turno,Salario
0,03232323P,María Luisa Galdón Ter,43534534567,noche,1200.0
1,04444444T,Úrsula Delta Camacho,11177788899,mañana,900.0
2,11111111Q,Luis Ramírez Pardo,23445556666,mañana,900.0
3,14111155T,Sergio Lérida Campos,55577700089,tarde,1000.0
4,14567555L,Piedad Colmenero Zapillo,23456734534,noche,1200.0


In [159]:
#guardando un csv de empleado
empleado.to_csv('empleado.csv', encoding='latin1')


**21. ¿Caules son los diferentes turnos que hay de trabajo?**

In [160]:
import numpy as np
empleado["Turno"].unique()

array(['noche', 'mañana', 'tarde'], dtype=object)

**22. Muestra un resumen general sobre los valores de la columna "salarios"**

In [161]:
empleado.describe()

,Salario
count,11.000000
mean,1027.272727
std,119.087439
min,900.000000
25%,950.000000
50%,1000.000000
75%,1100.000000
max,1200.000000


**23. ¿Cuantos trabajadores hay en cada turno?**

In [162]:
empleado['Turno'].value_counts()

Turno
tarde     5
noche     3
mañana    3
Name: count, dtype: int64

**24. ¿Cuales son los menus que tiene el restaurante, el codigo de cada uno y la cantidad de pedidos que tiene cada uno?**

In [163]:
menu = df14
menu.head()

,nombre,codigo,cantidad_pedidos
0,Menú de Hamburguesa con queso,12,6
1,Menú de Pollo con queso,13,6
2,Menú de Pollo,11,3
3,Queso,04,3
4,Patatas,02,2


**25. ¿Cuáles los últimos 5 registros de la tabla `EMPLEADO`?**

In [164]:
empleado = consultasql("select * from EMPLEADO")
print(empleado.tail())

          DNI                   Nombre          Nss   Turno  Salario
6   33333333M    Martín Guerrero López  33344455566   tarde   1000.0
7   55555555J     Carmen Hernández Pío  99966633311  mañana    900.0
8   77777777M       Pedro Jiménez Ruiz  23456785432   tarde   1000.0
9   88888888O  Soledad Campillo Molina  00088877754   noche   1200.0
10  99999999X        Raúl Rodrigo Roca  55566677789   tarde   1000.0


**26. ¿Cuál es el promedio del salario de los empleados**

In [170]:
empleado = consultasql("select * from EMPLEADO")
promedio_salario = empleado['Salario'].mean()
print(promedio_salario)

1027.2727272727273


**27. ¿ Cual es el menu que mas pedidos tiene?**

In [174]:
menu_mas_pedidos = menu.loc[menu['cantidad_pedidos'].idxmax()]
print(menu_mas_pedidos)


nombre              Menú de Hamburguesa con queso
codigo                                         12
cantidad_pedidos                                6
Name: 0, dtype: object


**28. Muestra los valores de la tabla `EMPLEADO` ordenados por salario de manera descendente**

In [173]:
empleado = consultasql("select * from EMPLEADO order by salario desc")
print(empleado)

          DNI                    Nombre          Nss   Turno  Salario
0   03232323P    María Luisa Galdón Ter  43534534567   noche   1200.0
1   14567555L  Piedad Colmenero Zapillo  23456734534   noche   1200.0
2   88888888O   Soledad Campillo Molina  00088877754   noche   1200.0
3   14111155T      Sergio Lérida Campos  55577700089   tarde   1000.0
4   22222222S         María Sánchez Cid  11112223334   tarde   1000.0
5   33333333M     Martín Guerrero López  33344455566   tarde   1000.0
6   77777777M        Pedro Jiménez Ruiz  23456785432   tarde   1000.0
7   99999999X         Raúl Rodrigo Roca  55566677789   tarde   1000.0
8   04444444T      Úrsula Delta Camacho  11177788899  mañana    900.0
9   11111111Q        Luis Ramírez Pardo  23445556666  mañana    900.0
10  55555555J      Carmen Hernández Pío  99966633311  mañana    900.0


**29. Lista el nombre de los empleados que comience con la letra 'A'.**

In [175]:
empleado = consultasql("select * from EMPLEADO where nombre like 'A%'")
print(empleado)

Empty DataFrame
Columns: [DNI, Nombre, Nss, Turno, Salario]
Index: []


**30. Muestra el salario máximo y mínimo de los empleados.**

In [177]:
empleado = consultasql("select * from EMPLEADO")
salario_max = empleado['Salario'].max()
salario_min = empleado['Salario'].min()
print(f"Salario máximo: {salario_max}, Salario mínimo: {salario_min}")

Salario máximo: 1200.0, Salario mínimo: 900.0
